In [1]:
#Importar librerías
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

#FUNCIONES DE SCRAPING
base_site = "http://books.toscrape.com/"
base_catalogue = "http://books.toscrape.com/catalogue/page-{}.html"
books_data = []

def get_rating_from_class(tag):
    ratings = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
    for c in tag.get("class", []):
        if c in ratings:
            return ratings[c]
    return None

def scrape_all_books():
    page = 1
    while True:
        url = base_catalogue.format(page)
        response = requests.get(url)
        if response.status_code != 200:
            break
        soup = BeautifulSoup(response.content, 'html.parser')
        books = soup.select('article.product_pod')
        if not books:
            break
        for book in books:
            title = book.h3.a['title']
            price = book.select_one('p.price_color').text
            availability = book.select_one('p.instock').text.strip()
            rating = get_rating_from_class(book.p)
            image_url = base_site + book.select_one('img')['src'].replace('../', '')
            detail_url = base_site + 'catalogue/' + book.h3.a['href'].replace('../../../', '')
            detail_response = requests.get(detail_url)
            detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
            breadcrumb = detail_soup.select('ul.breadcrumb li a')
            category = breadcrumb[2].text.strip() if len(breadcrumb) > 2 else "Desconocido"
            books_data.append({
                'Título': title,
                'Precio': price,
                'Disponibilidad': availability,
                'Rating': rating,
                'Categoría': category,
                'Imagen_URL': image_url
            })
        page += 1

# Ejecutar scraping
scrape_all_books()
df = pd.DataFrame(books_data)
df.to_csv('libros_completos.csv', index=False)
df.head()


,Título,Precio,Disponibilidad,Rating,Categoría,Imagen_URL
0,A Light in the Attic,£51.77,In stock,3,Poetry,http://books.toscrape.com/media/cache/2c/da/2c...
1,Tipping the Velvet,£53.74,In stock,1,Historical Fiction,http://books.toscrape.com/media/cache/26/0c/26...
2,Soumission,£50.10,In stock,1,Fiction,http://books.toscrape.com/media/cache/3e/ef/3e...
3,Sharp Objects,£47.82,In stock,4,Mystery,http://books.toscrape.com/media/cache/32/51/32...
4,Sapiens: A Brief History of Humankind,£54.23,In stock,5,History,http://books.toscrape.com/media/cache/be/a5/be...
